### QueryMyDoc.ipynb
Q&A your own documents using LangChain, GPT and Chroma DB as a vectorstrore
    
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rashlab/AI-Notes/blob/main/langchain/QueryMyDoc.ipynb)


In [ ]:
!pip install chromadb
!pip install langchain

In [37]:
import os
import requests
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

os.environ["OPENAI_API_KEY"] = "YOUR API KEY HERE"

In [40]:
# download the text file we will use - state_of_the_union.txt
input_file = 'state_of_the_union.txt'
if not os.path.exists(input_file):
    data_url = 'https://github.com/rashlab/AI-Notes/raw/main/langchain/state_of_the_union.txt'
    with open(input_file, 'w') as f:
        f.write(requests.get(data_url).text)

loader = TextLoader(input_file)

# Splitting the documents into chunks
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# using OpenAI embeddings, and create a vectorstore using Chroma DB
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embeddings)

# create the vectorstore index
retriever = db.as_retriever()

# we will compare two models - the 3.5 Turbo and the Davinci 003 
# we make sure to use temperature=0.0, so that the model will not add any noise to the output
llm_davinci003 = OpenAI(model_name="text-davinci-003", temperature=0.0)
llm_35Turbo = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

# create the QA chains - one for each model
# first we create a pair of QA chains w/o sources, and then a pair of QA chains with sources
# QA chains with sources will return the source document, but internally langchain also uses different prompt templates 
qa_davinci003 = RetrievalQA.from_chain_type(llm_davinci003, chain_type="stuff", retriever=retriever)
qa_35Turbo = RetrievalQA.from_chain_type(llm_35Turbo, chain_type="stuff", retriever=retriever)
qaWithSource_davinci003 = RetrievalQAWithSourcesChain.from_chain_type(llm_davinci003, chain_type="stuff", retriever=retriever)
qaWithSource_35Turbo = RetrievalQAWithSourcesChain.from_chain_type(llm_35Turbo, chain_type="stuff", retriever=retriever)

# we will use the following query, which is a bit tricky for the model...
query = "should we raise the corporate tax?"

Using embedded DuckDB without persistence: data will be transient
c:\Users\BASEMENT\.conda\envs\GPT\lib\site-packages\langchain\llms\openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\BASEMENT\.conda\envs\GPT\lib\site-packages\langchain\llms\openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [39]:
qa_davinci003.run(query)

" Yes, I've proposed a 15% minimum tax rate for corporations."

In [ ]:
qa_35Turbo.run(query)

In [ ]:
qaWithSource_davinci003({"question": query}, return_only_outputs=False)

In [ ]:
qaWithSource_35Turbo({"question": query}, return_only_outputs=False)